# Exercise 3: Time series 

Aim: To work with mooring data and plot time series

**Required Outputs:**
Three figures should be generated:
- ex3fig1-Solution-Messfern.png: Cosine fit to the CTD data and residuals
- ex3fig2-Solution-Messfern.png: Cosine fit to velocity data and the residuals
- ex3fig3-Solution-Messfern.png: A tidal ellipse
- ex3fig4-Solution-Messfern.png: The effect of filtering a time series

<hr>

**Complete your information here:**

- **Your Name:** [REPLACE WITH YOUR ACTUAL NAME]
- **Date:** [REPLACE WITH TODAY'S DATE]
- **Student ID:** [REPLACE WITH YOUR STUDENT ID]

**Replace the placeholder text above with your actual information.**

In [ ]:
# Importing required packages here
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import scipy.signal as sg
from scipy.optimize import curve_fit
import sys

import gsw


import_paths = ['.', '..']
modules_imported = False

for path in import_paths:
    try:
        sys.path.append(path)
        from modules import tools
        modules_imported = True
        print(f"✓ Modules imported from: {path}")
        break
    except ImportError:
        continue

if not modules_imported:
    # Fallback: import functions directly
    print("⚠️  Using fallback imports")
    from modules import tools

In [ ]:
# Set paths relative to the current notebook location
datadir = '../data/'

# SOLUTION: Output file paths for figures
figdir = '../figures/'

# Create figures directory if it doesn't exist
if not os.path.exists(figdir):
    os.makedirs(figdir)
    
print(f"Figures will be saved to: {figdir}")
print(f"Data directory: {datadir}")

In [ ]:
# Test that setup is complete
assert 'figdir' in locals(), "figdir variable not defined"
assert figdir.endswith('/'), "figdir should end with '/'"
# Note: datadir will be created by download function, so we don't test for existence here
assert 'datadir' in locals(), "datadir variable not defined"
print("✓ File setup test passed!")

# Load the CTD data

In [ ]:
# Specify the input file
inputfile = os.path.join(datadir, 'mooredCTD1_raw.nc')

# Load the input file
ctd_data = xr.open_dataset(inputfile)
# print(ctd_data)


### Make a simple time series plot of salinity (Do not save)

- Note that the axes limits are not ideal because of the large changes in salinity at the beginning and end of the time series

In [ ]:
# Plot the time series of PSAL
plt.figure(figsize=(15, 5))
plt.plot(ctd_data.time, ctd_data.PSAL, label=f'Mooring 1')

plt.xlabel('Time')
plt.ylabel('Salinity (PSU)')
plt.title('Time Series of Salinity (PSAL)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compute SA from PSAL and CT from TEMP
ctd_data['SA'] = # TODO: Add your code here to compute Absolute Salinity from PSAL
ctd_data['CT'] = gsw.CT_from_t(ctd_data.SA, ctd_data.TEMP, ctd_data.PRES)

# plot the time series of SA and PSAL from mooring 1
# TODO: Play around with the figure size until you get legible axes
plt.figure(figsize=(22, 1))
plt.plot(ctd_data.time, ctd_data.SA, label='SA')
plt.plot(ctd_data.time, ctd_data.PSAL, label='PSAL')
plt.xlabel('Time')
plt.ylabel('Salinity (PSU)')
plt.title('Time Series of Salinity (PSAL) and Absolute Salinity (SA) from Mooring 1')
plt.legend()
plt.grid(True)

# Set y-axis limits to the middle 99% of the PSAL and SA values
# Calculate the 0.5th and 99.5th percentiles for PSAL and SA
psal_005 = ctd_data.PSAL.quantile(0.005)
psal_995 = # TODO: Add your code here to compute the 99.5th percentile for PSAL

# Set y-axis limits to the middle 95% of the PSAL and SA values
plt.ylim(np.floor(psal_005) - 0.1, np.ceil(psal_995) + 0.1)

plt.show()

# Fit a tidal harmonic

Since the region has strong tidal variability, let's see if we can extract the tidal cycle.  

First we will work with the time coordinate to make it in units of fractional days.  The cell below makes both fractional and integer days.

**What might be the problem with using integer days?**


In [ ]:
# Extract time in days since the start
time_in_days1 = (ctd_data.time - ctd_data.time[0]).astype('timedelta64[s]') / np.timedelta64(1, 's') / 86400
time_in_days2 = (ctd_data.time - ctd_data.time[0]).astype('timedelta64[D]') / np.timedelta64(1, 'D')

print(f"Min of time_in_days1: {time_in_days1.min().values}")
print(f"Max of time_in_days1: {time_in_days1.max().values}")

print(f"Time interval: {time_in_days1.min().values} to {time_in_days1.max().values}")
time_diff = np.diff(time_in_days1) 
print(time_diff)

# Turn on this plot to see how the time differs between time_in_days1 and time_in_days2
if 0:
    # Plot the two time representations against the original time in separate subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

    # Plot time in days (seconds conversion)
    ax1.plot(ctd_data.time, time_in_days1, label='Time in days (seconds conversion)')
    ax1.set_ylabel('Time in Days (seconds conversion)')
    ax1.set_title('Time in Days (seconds conversion)')
    ax1.legend()
    ax1.grid(True)

    # Plot time in days (days conversion)
    ax2.plot(ctd_data.time, time_in_days2, label='Time in days (days conversion)')
    ax2.set_xlabel('Original Time')
    ax2.set_ylabel('Time in Days (days conversion)')
    ax2.set_title('Time in Days (days conversion)')
    ax2.legend()
    ax2.grid(True)
    # Angle the x-axis labels for better readability
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=30, ha='right')
    plt.show()

# Fitting a harmonic

Read about [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) from the `scipy.optimize` library: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

Below, there is a function to calculate a semidiurnal cosine fit.  It's written for a semi-diurnal solar tide (12 hours).

- Edit the function so that it uses the M2 tide
- Move the function into the modules/tools.py file 
- Add any necessary package imports at the top of tools py.
- Import the function in this cell using `from modules.tools import semi_diurnal_cosine`

For an example of what a module file might look like, look in your exercise-02-bathymetry/modules/bathymetry.py 

In [ ]:
# Define the cosine function with a period of the S2 lunar tide (12 hours)
def semi_diurnal_cosine(t, amplitude, phase, offset):
    period = 12 / 24  # Convert hours to days
    return amplitude * np.cos(2 * np.pi * t / period + phase) + offset

# TODO: Update this function for the M2 lunar tide and MOVE the function into modules/tools.py
# Then you will need to: restart your kernel and import the function here as
# from modules.tools import semi_diurnal_cosine
# You will get an error because at the top of tools.py, you'll also need to import the numpy package as np

# Extract salinity data for a specific mooring (e.g., Mooring 1)
salinity_data = ctd_data.PSAL.values

# Choose which of the two time_in_days above to use
time_in_days = time_in_days1

# Fit the semi-diurnal cosine function to the salinity data
# Remove points which are not finite from the time_in_days and salinity_data vectors
finite_indices = np.isfinite(time_in_days) & np.isfinite(salinity_data)
time_in_days = time_in_days[finite_indices]
salinity_data = salinity_data[finite_indices]

# Remove salinity values below 19
valid_indices = salinity_data >= 19
time_in_days = time_in_days[valid_indices]
salinity_data = salinity_data[valid_indices]

# Read about scipy.optimize curve_fit function https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
popt, pcov = curve_fit(semi_diurnal_cosine, time_in_days, salinity_data, p0=[1, 0, np.mean(salinity_data)])

# Extract the fitted parameters
amplitude, phase, offset = popt

# Generate the fitted salinity values
fitted_salinity = semi_diurnal_cosine(time_in_days, amplitude, phase, offset)

# TODO: Calculate the residual (difference between original and fitted data), i.e. subtract the fitted data from the original
residual_salinity = # Add your code here

# TODO: Create a figure showing both the fit and the residuals
# Plot the original and fitted salinity data
plt.figure(figsize=(10, 8))

# TODO: Add subplot 1 - original data and fitted curve
plt.subplot(2, 1, 1)
# Add your plotting code here

# TODO: Add subplot 2 - residuals
plt.subplot(2, 1, 2)
# Add your plotting code here

plt.tight_layout()
plt.show()

# TODO: Save this figure using the naming convention from previous exercises

# Load the velocity time series

Uncomment the plotting code in the next cell.
- What can you say about the relative magnitudes of velocity in these two directions?

In [ ]:
# Define the path to the file
velo_file = os.path.join(datadir, 'mooring1velocity.nc')

# Open the dataset
velo_data = xr.open_dataset(velo_file)

# You can print a list of variables to see what they're called
print(list(velo_data.data_vars))

# Change the 0 to 1 to turn on the plotting
if 1:
    # Plot the time series of UVEL, VVEL, and WVEL
    plt.figure(figsize=(10, 5))

    # Plot UVEL
    plt.subplot(2, 1, 1)
    plt.plot(velo_data.time, velo_data.UVEL, label='UVEL')
    plt.xlabel('Time')
    plt.ylabel('U Velocity (m/s)')
    plt.title('Time Series of U Velocity (UVEL)')
    plt.legend()
    plt.grid(True)

    # Plot VVEL
    plt.subplot(2, 1, 2)
    # TODO: Add a line here to plot the V velocity data
    plt.xlabel('Time')
    plt.ylabel('V Velocity (m/s)')
    plt.title('Time Series of V Velocity (VVEL)')
    plt.legend()
    plt.grid(True)


    plt.tight_layout()
    plt.show()



## Fit a tidal harmonic to the velocity data

In [ ]:
# Extract UVEL and VVEL data
uvel_data = velo_data.UVEL.values
vvel_data = velo_data.VVEL.values

# Use the same time_in_days as before
time_in_days = (velo_data.time - velo_data.time[0]).astype('timedelta64[s]') / np.timedelta64(1, 's') / 86400

# Remove points which are not finite from the time_in_days and velocity data vectors
finite_indices_uvel = np.isfinite(time_in_days) & np.isfinite(uvel_data)
finite_indices_vvel = np.isfinite(time_in_days) & np.isfinite(vvel_data)

time_in_days_uvel = time_in_days[finite_indices_uvel]
uvel_data = uvel_data[finite_indices_uvel]

time_in_days_vvel = time_in_days[finite_indices_vvel]
vvel_data = vvel_data[finite_indices_vvel]

# Fit the semi-diurnal cosine function to the UVEL data
popt_uvel, _ = curve_fit(semi_diurnal_cosine, time_in_days_uvel, uvel_data, p0=[1, 0, np.mean(uvel_data)])
amplitude_uvel, phase_uvel, offset_uvel = popt_uvel
fitted_uvel = # Calculate the fitted velocity data

# Fit the semi-diurnal cosine function to the VVEL data
popt_vvel, _ = # TODO: Calculate the semidiurnal fit for v-velocity data
amplitude_vvel, phase_vvel, offset_vvel = # TODO: Split the tuple results into individual variables
fitted_vvel = # Calculate the fitted velocity data

# TODO: Calculate residuals for both UVEL and VVEL
residual_uvel = # Add your code here
residual_vvel = # Add your code here

# TODO: Create a figure showing the fits and residuals for velocity data
# Plot the original and fitted UVEL and VVEL data with residuals
plt.figure(figsize=(10, 8))
plt.subplot(4, 1, 1)

plt.subplot(4, 1, 1)
# 1. UVEL data and fit
plt.plot(time_in_days_uvel, uvel_data, label='Original UVEL Data')
plt.plot(time_in_days_uvel, fitted_uvel, label='Fitted Semi-Diurnal Cosine', linestyle='--')
#plt.xlabel('Time (days)')
plt.ylabel('U Velocity (m/s)')
plt.title('Fitting Semi-Diurnal Cosine to U Velocity (UVEL)')
plt.legend()
plt.grid(True)

# TODO: Add 3 more subplots showing:
# 2. UVEL residuals
# 3. VVEL data and fit  
# 4. VVEL residuals

plt.tight_layout()
plt.show()

# TODO: Save this figure following the naming convention 

# TODO: What happens if you omit (comment out) the lines of code starting with `finite_indices_uvel`? 
# Does the code break? Can you read the error message to understand why?

# Plotting a tidal ellipse

Add the original data to the plot as a scatter (see `ax1.scatter()`)

In [ ]:
# Generate time values for one period (12 hours - but update this for the M2 period)
period_hours = 12
period_days = period_hours / 24
time_values = np.linspace(0, period_days, 1000)

# Generate fitted uvel and vvel values using the fitted parameters
fitted_uvel_values = semi_diurnal_cosine(time_values, amplitude_uvel, phase_uvel, offset_uvel)
fitted_vvel_values = # TODO: Calculate the vvel data for this time vector

# Plot the ellipse and add the original data as scatter points
fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot the ellipse with square axes
ax1.plot(fitted_uvel_values, fitted_vvel_values, label='Fitted Ellipse')

# TODO: Add the original data as scatter points
# ax1.scatter(...)

ax1.set_xlabel('U Velocity (m/s)')
ax1.set_ylabel('V Velocity (m/s)')
ax1.set_title('Tidal Ellipse and Original Data')
ax1.legend()
ax1.grid(True)
ax1.axis('equal')

plt.show()

# TODO: Save this figure



# Filter data

Sometimes we would like to low-pass filter (remove the high frequency variability) by using a running average (also called a moving average, or convolving with a boxcar filter). See below for an example.
- Try varying the Npoints (make it 2*Npoints or 0.5*Npoints) and note the effect


In [ ]:
# Apply a 1-day boxcar filter to the data variables SA and CT
# Calculate the number of samples in 24 hours
time_diff = np.diff(ctd_data.time.values) / np.timedelta64(1, 's')
time_diff_seconds = np.mean(time_diff)
print(f"Average time difference in seconds: {time_diff_seconds}")

print(f"Time difference in seconds: {time_diff_seconds}")
samples_per_day = int(24 * 3600 / time_diff_seconds)
print(f"Number of samples in 24 hours: {samples_per_day}")

Npoints = samples_per_day
Npoints = int(Npoints)
time_filtered = time_in_days1.rolling(time=Npoints, center=True).mean()
ctd_data['SA_filtered'] = ctd_data.SA.rolling(time=Npoints, center=True).mean()


# Plot the original and filtered time series of SA from mooring 1
plt.figure(figsize=(12, 6))

# Plot SA
plt.subplot(2, 1, 1)
plt.plot(ctd_data.SA, label='Original SA')
# TODO: Add your code here to plot the filtered SA
# plt.plot(...)

plt.xlabel('Time')
plt.ylabel('Absolute Salinity (SA) (PSU)')
plt.title('Time Series of Absolute Salinity (SA) from Mooring 1')
plt.legend()
plt.grid(True)
plt.ylim(19, 30)

plt.tight_layout()
plt.show()

# TODO: Save this figure showing the effect of filtering

## Verify that the filter is doing what you expect

In [ ]:
# Manually average the data for a specific date

# Define the time range
start_date = '2024-05-22'
end_date = '2024-05-22'

# Select the data within the specified time range for mooring=1
sa_data_range = ctd_data.SA.sel(time=slice(start_date, end_date))
SA_filtered_range = ctd_data.SA_filtered.sel(time=slice(start_date, end_date))

# Manual average of SA
average_sa = sa_data_range.mean().values

# Plot the Absolute Salinity (SA) data within the specified time range
plt.figure(figsize=(15, 5))
plt.plot(sa_data_range.time, sa_data_range, label=f'Mooring 1')
plt.axhline(y=average_sa, color='r', linestyle='--', label='Average SA')
plt.plot(SA_filtered_range.time, SA_filtered_range, linestyle='--', color='green', label='Filtered SA')
plt.xlabel('Time')
plt.ylabel('Absolute Salinity (SA) (PSU)')
plt.title(f'Time Series of Absolute Salinity (SA) between {start_date} and {end_date}')
plt.legend()
plt.grid(True)
plt.show()

# If correct, average SA (the horizontal line) should intersect the filtered SA (the green line) at the midpoint of the day.

## Analysis Questions

**Question 1:** What is the period of the M2 lunar tide used in your harmonic fitting?  Express this in both hours and days.

**Question 2:** Looking at your residuals plots, do you think a single harmonic (M2 tide) adequately captures the periodic variability in the data?  If not, then what other tidal constituents might be present?

**Question 3:** What does the ellipse tell you about the tidal flow at this mooring location?
